In [1]:
from nbodykit.lab import *
import pandas as pd
import numpy as np

zmin, zmax = 0.01, 0.15

# Load your data and randoms as Pandas DataFrames
sdss = pd.read_csv('../data/sdss_zmin_0.000_zmax_0.300_sigma_5.0.csv')
sdss  = sdss[(sdss["red"] > zmin)&(sdss["red"] < zmax)]
randoms = pd.read_csv('../data/random_sample_healpy_128.csv')  # Replace with your randoms CSV file path

# Convert DataFrames to ArrayCatalog with RA, DEC, and Redshift
data_cat = ArrayCatalog({'RA': sdss['ra'].values,
                         'DEC': sdss['dec'].values,
                         'Redshift': sdss['red'].values})

rand_cat = ArrayCatalog({'RA': randoms['ra'].values,
                         'DEC': randoms['dec'].values,
                         'Redshift': randoms['red'].values})

data_cat['RA'] = np.radians(data_cat['RA'])
data_cat['DEC'] = np.radians(data_cat['DEC'])
rand_cat['RA'] = np.radians(rand_cat['RA'])
rand_cat['DEC'] = np.radians(rand_cat['DEC'])

# Assign cosmology (if not already in the data)
cosmo = cosmology.Planck15

# Define the edges for s and mu bins
s_bins = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]  # smaller range for simplicity
mu_bins = 10  # fewer bins for testing

# Define the SurveyData2PCF object
result = SurveyData2PCF('2d', data_cat, rand_cat, edges=s_bins, Nmu=mu_bins, cosmo=cosmo, ra='RA', dec='DEC', redshift='Redshift')

# Access the computed correlation function
xi_s_mu = result.corr.data  # 2D correlation function values
s = result.corr.coords['r']  # The s bins
mu = result.corr.coords['mu']  # The mu bins

# Save or print results
print("s:", s)
print("mu:", mu)
print("xi(s, mu):", xi_s_mu)

# Optionally save the results to a file
np.savez('xi_s_mu_results.npz', s=s, mu=mu, xi_s_mu=xi_s_mu)



s: [15. 25. 35. 45. 55. 65. 75. 85. 95.]
mu: [0.05 0.15 0.25 0.35 0.45 0.55 0.65 0.75 0.85 0.95]
xi(s, mu): [[(-6.54856454e-03, 11.8863675 ) (-6.49040742e-03, 11.924057  )
  (-7.03504066e-03, 12.00622028) (-8.49306114e-03, 12.14944911)
  (-9.45146483e-03, 12.36973511) (-9.00030168e-03, 12.68225391)
  (-7.82155124e-03, 13.08300176) (-1.06232706e-02, 13.58780181)
  (-1.84417422e-02, 14.16851866) ( 6.69843958e-03, 15.2844267 )]
 [( 1.00000000e+00,  0.        ) ( 4.42619700e-01,  0.        )
  ( 2.69010608e-01, 20.10048597) ( 9.55745739e-02, 20.2381846 )
  ( 1.66877587e-02, 20.35884521) (-2.38370861e-02, 20.61067717)
  (-5.48801423e-03, 21.09565753) (-2.88684119e-03, 21.94488449)
  (-8.21231762e-03, 23.29543221) ( 9.41544481e-04, 25.03266897)]
 [(            nan,  0.        ) (            nan,  0.        )
  (            nan,  0.        ) (            nan,  0.        )
  (            nan,  0.        ) (            nan,  0.        )
  (            nan,  0.        ) ( 2.08539678e-01, 30.3360

/home/fdavilakurban/anaconda3/envs/nbodykit/lib/python3.6/site-packages/nbodykit/algorithms/paircount_tpcf/estimators.py:229: UserWarning: The RR calculation in the Landy-Szalay estimator contains separation bins with no bins. This will result in NaN values in the resulting correlation function. Try increasing the number of randoms and/or using broader bins.
  warnings.warn(msg)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load the results (if saved) or use the variables directly
# data = np.load('xi_s_mu_results.npz')
# s = data['s']
# mu = data['mu']
# xi_s_mu = data['xi_s_mu']

# Ensure s and mu are arrays, not edges
s_centers = 0.5 * (s[1:] + s[:-1])  # Bin centers for s
mu_centers = mu  # Mu is already centered if Nmu was provided

# Create a 2D grid for plotting
s_grid, mu_grid = np.meshgrid(s_centers, mu_centers, indexing='ij')

# Plotting
plt.figure(figsize=(8, 6))
plt.pcolormesh(mu_grid, s_grid, xi_s_mu.T, shading='auto', cmap='viridis')
plt.colorbar(label=r'$\xi(s, \mu)$')
plt.xlabel(r'$\mu$ (cosine of line of sight angle)')
plt.ylabel(r'$s$ (Mpc/h)')
plt.title('2D Correlation Function: $\xi(s, \mu)$')
plt.tight_layout()
plt.show()
